In [1]:
import pyodbc
import mysql.connector
import dask.dataframe as dd
import pandas as pd
# from scripts.db_utils import create_ebay_connection
import os 
from dotenv import load_dotenv

load_dotenv()

def create_ebay_connection():
    """Create connection to the hosted SQL Server database for IP addresses."""
    try:
        connection = pyodbc.connect(
            f"DRIVER={{MySQL ODBC 9.1 Unicode Driver}};"
            f"SERVER={os.getenv('EBAY_DB_HOST')};"
            f"DATABASE={os.getenv('EBAY_DB_NAME')};"
            f"UID={os.getenv('EBAY_DB_USER')};"
            f"PWD={os.getenv('EBAY_DB_PASSWORD')}"
        )
        return connection
    except pyodbc.Error as e:
        print(f"Error connecting to hosted database: {e}")
        return None
# Database connection parameters
conn = create_ebay_connection()

query = "SELECT * FROM tbl_iptrace_user_activity where ams_trans_rsn_cd=0"




def fetch_data_in_chunks(conn, query, chunk_size=100000):
    """
    Fetch data from the database in chunks, then convert to Dask DataFrame.
    """
    offset = 0
    while True:
        # Modify the query to include pagination using LIMIT and OFFSET for chunks
        paginated_query = f"{query} LIMIT {chunk_size} OFFSET {offset}"
        
        # Fetch data in chunks using pandas (it handles column and row alignment internally)
        df_chunk = pd.read_sql_query(paginated_query, conn)
        
        if df_chunk.empty:
            break
        
        # Convert to Dask DataFrame
        dask_df = dd.from_pandas(df_chunk, npartitions=4)
        
        yield dask_df
        
        offset += chunk_size


dask_chunks = []

# Fetch chunks and store them in the list
for dask_df_chunk in fetch_data_in_chunks(conn, query, chunk_size=100000):
    dask_chunks.append(dask_df_chunk)

# Concatenate all chunks into a single Dask DataFrame
full_dask_df = dd.concat(dask_chunks)

# Now you can perform operations on full_dask_df
# result = full_dask_df.groupby('some_column').agg({'another_column': 'mean'})

# Compute the result
# computed_result = result.compute()

# Show the result
print(full_dask_df.head())


C:\Users\ramegupta\AppData\Local\Temp\ipykernel_15936\3663271736.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(paginated_query, conn)
C:\Users\ramegupta\AppData\Local\Temp\ipykernel_15936\3663271736.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(paginated_query, conn)
C:\Users\ramegupta\AppData\Local\Temp\ipykernel_15936\3663271736.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(paginated_query, conn)
C:\Users\ramegupta\

   id     CLNT_RMT_IP  ROI_CLICK_EVENT_TS  \
0   1  77.111.247.168 2020-08-02 18:37:01   
1   2   66.249.73.101 2020-08-21 03:36:38   
2   3  77.111.247.129 2020-08-21 03:36:38   
3   4   66.249.73.101 2020-08-21 03:36:33   
4   5  77.111.247.129 2020-08-21 03:36:33   

                                          BRWSR_NAME  ams_trans_rsn_cd  \
0  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
1  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
2  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
3  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
4  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   

  ams_pblshr_id    buyer_id  is_vpn            click_id  
0    5574672411   valen_949       1  209245484389035777  
1    5574672411  morbanisaf       1  209453736892211586  
2    5574672411  morbanisaf       1  209453736892211586  
3    5574672411  morbanisaf       1  209453736892211

In [3]:
# Get the shape of the DataFrame (number of rows and columns)
num_rows, num_columns = full_dask_df.shape[0].compute(), full_dask_df.shape[1]

print(f"Rows: {num_rows}, Columns: {num_columns}")


Rows: 1254243, Columns: 9


In [1]:
import dask.dataframe as dd
import pandas as pd


data_dict = {
    "id": [1, 2, 3, 4, 5],
    "CLNT_RMT_IP": ["77.111.247.168", "66.249.73.101", "77.111.247.129", "66.249.73.101", "77.111.247.129"],
    "ROI_CLICK_EVENT_TS": ["2020-08-02 18:37:01", "2020-08-21 03:36:38", "2020-08-21 03:36:38", "2020-08-21 03:36:33", "2020-08-21 03:36:33"],
    "BRWSR_NAME": [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36"
    ],
    "ams_trans_rsn_cd": [0, 0, 0, 0, 0],
    "ams_pblshr_id": ["5574672411", "5574672411", "5574672411", "5574672411", "5574672411"],
    "buyer_id": ["valen_949", "morbanisaf", "morbanisaf", "morbanisaf", "morbanisaf"],
    "is_vpn": [1, 1, 1, 1, 1],
    "click_id": ["209245484389035777", "209453736892211586", "209453736892211586", "209453736892211586", "209453736892211586"]
}
pandas_df = pd.DataFrame(data_dict)

# Convert the Pandas DataFrame to a Dask DataFrame
dask_df = dd.from_pandas(pandas_df, npartitions=1)

# Display the Dask DataFrame
print(dask_df)


Dask DataFrame Structure:
                  id CLNT_RMT_IP ROI_CLICK_EVENT_TS BRWSR_NAME ams_trans_rsn_cd ams_pblshr_id buyer_id is_vpn click_id
npartitions=1                                                                                                         
0              int64      string             string     string            int64        string   string  int64   string
4                ...         ...                ...        ...              ...           ...      ...    ...      ...
Dask Name: frompandas, 1 expression
Expr=df


In [2]:
dask_df.head()

,id,CLNT_RMT_IP,ROI_CLICK_EVENT_TS,BRWSR_NAME,ams_trans_rsn_cd,ams_pblshr_id,buyer_id,is_vpn,click_id
0,1,77.111.247.168,2020-08-02 18:37:01,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0,5574672411,valen_949,1,209245484389035777
1,2,66.249.73.101,2020-08-21 03:36:38,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0,5574672411,morbanisaf,1,209453736892211586
2,3,77.111.247.129,2020-08-21 03:36:38,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0,5574672411,morbanisaf,1,209453736892211586
3,4,66.249.73.101,2020-08-21 03:36:33,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0,5574672411,morbanisaf,1,209453736892211586
4,5,77.111.247.129,2020-08-21 03:36:33,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0,5574672411,morbanisaf,1,209453736892211586


In [3]:
unique_ips_per_buyer = dask_df.groupby('buyer_id')['CLNT_RMT_IP'].nunique().compute()

# Display the result
print(unique_ips_per_buyer)


buyer_id
valen_949     1
morbanisaf    2
Name: CLNT_RMT_IP, dtype: int64


In [4]:
# Total click count from each IP
total_clicks_per_ip = dask_df.groupby('CLNT_RMT_IP')['click_id'].count().compute()
print("Total clicks per IP:")
print(total_clicks_per_ip)


Total clicks per IP:
CLNT_RMT_IP
66.249.73.101     2
77.111.247.129    2
77.111.247.168    1
Name: click_id, dtype: int64


In [7]:
dask_df.columns

Index(['id', 'CLNT_RMT_IP', 'ROI_CLICK_EVENT_TS', 'BRWSR_NAME',
       'ams_trans_rsn_cd', 'ams_pblshr_id', 'buyer_id', 'is_vpn', 'click_id'],
      dtype='object')

In [5]:
# Find the total click count per user
clicks_per_user = dask_df.groupby('buyer_id').size().compute()

# Display the result
print(clicks_per_user)


buyer_id
morbanisaf    4
valen_949     1
dtype: int64


## Unique IPs associated with each user(for each publisher) and their click counts

In [6]:
# Combine multiple operations before computing
result = dask_df.groupby('buyer_id')['CLNT_RMT_IP'].nunique().to_frame(name='unique_ips') \
                .merge(dask_df.groupby('buyer_id').size().to_frame(name='total_clicks'), 
                       left_index=True, right_index=True)

# Trigger computation once
result = result.compute()
print(result)


            unique_ips  total_clicks
buyer_id                            
valen_949            1             1
morbanisaf           2             4


In [9]:
# Find the number of clicks per publisher, per user, and per IP
clicks_per_publisher_user_ip = dask_df.groupby(['ams_pblshr_id', 'buyer_id', 'CLNT_RMT_IP']).size().compute()

# Display the result
print(clicks_per_publisher_user_ip)



ams_pblshr_id  buyer_id    CLNT_RMT_IP   
5574672411     morbanisaf  66.249.73.101     2
                           77.111.247.129    2
               valen_949   77.111.247.168    1
dtype: int64


In [11]:
# Group by buyer_id and count the number of unique publishers
users_multiple_publishers = dask_df.groupby('buyer_id')['ams_pblshr_id'].nunique()

# Filter users who have more than one unique publisher
users_multiple_publishers = users_multiple_publishers[users_multiple_publishers > 1]

# Compute the result
result = users_multiple_publishers.compute()
print(result)


Series([], Name: ams_pblshr_id, dtype: int64)


In [12]:
# Group by buyer_id and ams_pblshr_id to check the unique combinations
user_publisher_counts = dask_df.groupby('buyer_id')['ams_pblshr_id'].nunique()

# Get users with more than one unique publisher
users_with_multiple_publishers = user_publisher_counts[user_publisher_counts > 1]

# Compute the result
result = users_with_multiple_publishers.compute()
print(result)


Series([], Name: ams_pblshr_id, dtype: int64)
